<a href="https://colab.research.google.com/github/keivanipchihagh/intro_to_machineLearning/blob/master/Deep_Learning_with_Python_Fran%C3%A7ois_Chollet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **2.1 - Digit Classification With MNIST Dataset**

In [0]:
from keras.datasets import mnist  # MNIST is a preloaded dataset in keras
import keras
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()  # load mnist data as numpy arrays

# Dataset description
print('Dimensions: ', train_images.shape)
print('Length: ', len(train_labels))
print('Labels: ', train_labels)

# Create, initialize & compile the network
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, activation = keras.activations.relu, input_shape = (28 * 28,)))
model.add(keras.layers.Dense(10, activation = keras.activations.softmax))
model.compile(optimizer = keras.optimizers.rmsprop(lr = 0.001), loss = keras.losses.categorical_crossentropy, metrics = [keras.metrics.accuracy])
model.summary();

# Prepare the data
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Train the model
model.fit(train_images, train_labels, epochs = 5, batch_size = 128)

# Test the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test Accuracy: ', test_acc)

**3.1.3 - Loss functions and optimizers: keys to configuring the learning process**

Fortunately, when it comes to common problems such as classification regression, and sequence prediction, there are simple guidelines you can follow to choose the correct loss. For instance, you’ll use **binary crossentropy** for a **two-class classification problem**, **categorical crossentropy** for a **many-class classification problem**, **meansquared error** for a **regression problem**, **connectionist temporal classification (CTC)** for a **sequence-learning problem**, and so on.

## **3.4 - Movie Classification with IMDB Dataset**

In [0]:
from keras.datasets import imdb # IMDB is a preloaded dataset in keras
import numpy as np
import keras
import matplotlib.pyplot as plt

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000); # Load the top 10000 frequently data into numpy arrays
print("Training data: ", train_data[0])  # Each word has been assosiated with a corresponding integer

def decode(index):  # Decoding the sequential integers into the corresponding words
  word_index = imdb.get_word_index()
  reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
  decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[index]])
  return decoded_review

print("Decoded training data: ", decode(0))

def vectorize_sequences(sequences, dimension = 10000):  # Encoding the integer sequences into a binary matrix
  results = np.zeros((len(sequences), dimension)) # Creating an all-zero matrix
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1.
  return results

train_data = vectorize_sequences(train_data)
test_data = vectorize_sequences(test_data)

print("Vectorized training data: ", train_data, sep = '\n');

# Convert to floats
train_labels = np.asarray(train_labels).astype('float32')
test_labels = np.asarray(test_labels).astype('float32')

# Create, initialize & compile the network
model = keras.models.Sequential()
model.add(keras.layers.Dense(16, activation = 'relu', input_shape = (10000,)))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer = keras.optimizers.RMSprop(lr = 0.001), loss = keras.losses.binary_crossentropy, metrics = [keras.metrics.binary_accuracy, 'acc'  ])
model.summary();

# Set 10000 records as validation data
x_val = train_data[:10000]
train_data = train_data[10000:]
y_val = train_labels[:10000]
train_labels = train_labels[10000:]

history = model.fit(train_data, train_labels, epochs = 5, batch_size = 512, validation_data = (x_val, y_val))  # Train the model and get results

# Draw the statistice for the training process
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'], 'bo', label = 'Training loss')
plt.plot(epochs, history.history['val_loss'], 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.clf()
plt.plot(epochs, history.history['acc'], 'bo', label='Training acc')
plt.plot(epochs, history.history['val_acc'], 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

model.evaluate(test_data, test_labels);

**3.4.2 - Encoding the integer sequences into a binary matrix**

You can’t feed lists of integers into a neural network. You have to turn your lists into tensors. There are two ways to do that:

*   One-hot encode your lists to turn them into vectors of 0s and 1s. This would mean, for instance, turning the sequence [3, 5] into a 10,000-dimensional vector that would be all 0s except for indices 3 and 5, which would be 1s. Then you could use as the first layer in your network a Dense layer, capable of handling floating-point vector data. **(Used)**

*   Pad your lists so that they all have the same length, turn them into an integer
tensor of shape (samples, word_indices), and then use as the first layer in
your network a layer capable of handling such integer tensors (the Embedding
layer, which we’ll cover in detail later in the book).

## **3.5 - Newswires Classification with Reuters Dataset**

In [0]:
from keras.datasets import reuters # Reuters comes preloaded with keras
from keras.utils.np_utils import to_categorical
import numpy as np
import keras
from keras import regularizers
import matplotlib.pyplot as plt

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = 10000) # Load top 10000 wrods dataset

print("Test data sample: ", test_data[0])  # Each word has been assosiated with a corresponding integer

def decode(index):  # Decoding the sequential integers into the corresponding words
  word_index = reuters.get_word_index()
  reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
  decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in test_data[0]])
  return decoded_newswire

print("Decoded test data sample: ", decode(0))

def vectorize_sequences(sequences, dimension = 10000):    # Encoding the integer sequences into a binary matrix
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1.
  return results

train_data = vectorize_sequences(train_data)
test_data = vectorize_sequences(test_data)

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model = keras.models.Sequential();
model.add(keras.layers.Dense(64, activation = 'relu', input_shape = (10000, )))
model.add(keras.layers.Dense(64, activation = 'relu'))
model.add(keras.layers.Dense(46, activation = 'softmax')) # Softmax is used to output probability distribution verctor which sum to 1
model.compile( optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

x_val = train_data[:1000]
train_data = train_data[1000:]
y_val = train_labels[:1000]
train_labels = train_labels[1000:]

history = model.fit(train_data, train_labels, batch_size = 512, epochs = 10, validation_data = (x_val, y_val), verbose = False)

epochs = range(1, len(history.history['loss']) + 1)

plt.plot(epochs, history.history['loss'], 'bo', label = 'Training loss')
plt.plot(epochs, history.history['val_loss'], 'b', label = 'Validation loss')
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
plt.plot(epochs, history.history['accuracy'], 'bo', label='Training acc')
plt.plot(epochs, history.history['val_accuracy'], 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluation
results = model.evaluate(test_data, test_labels)
print(results)

# Predictions
predictions = model.predict(test_data)
print('Prediction index: ', np.argmax(predictions[0]))

**Over fitting the weights**

One major problem during the traning process is *over fitting*. There are multiple ways to avoid or optimize this. But simple enough, by analysing the plots you may notice unexpected changes. like, From a certain epochs, the validation results tend to slow down or the slope decreases. That's your sign of overfitting!

## **3.6 - Predicting house prices: a regression example with BHP dataset**

In [0]:
from keras.datasets import boston_housing
import keras, numpy as np, matplotlib.pyplot as plt

(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

print("Raw traning data: \n", train_data, end = '\n\n');

# Normalazing the data
mean = train_data.mean(axis = 0)
train_data -= mean
std = train_data.std(axis = 0)
train_data /= std
test_data -= mean
test_data /= std

print("Normalized traning data: \n", train_data)

def build_model():  # Create, initialize and compile the network
  model = keras.models.Sequential()
  model.add(keras.layers.Dense(64, activation = 'relu', input_shape = (training_data[1], )))
  model.add(keras.layers.Dense(64, activation = 'relu'))
  model.add(keras.layers.Dense(1))  # If activation method is used, the output will be constrained to a certain range
  model.compile(optimizer = 'rmsprop', losss = 'mse', metrics = ['mae'])
  return model



**3.6.2 Preparing the data**

It would be problematic to feed into a neural network values that all take wildly different ranges. The network might be able to automatically adapt to such heterogeneous data, but it would definitely make learning more difficult. A widespread best practice to deal with such data is to do **feature-wise normalization**: **for each feature in the input data (a column in the input data matrix), you subtract the mean of the feature and divide by the standard deviation, so that the feature is centered around 0 and has a unit standard deviation.** This is easily done in Numpy.

# **4.2 - K Fold Cross-Validation**

In [0]:
import numpy as np

k = 4
num_validation_samples = len(data) // k
np.random.shuffle(data)
validation_scores = []

for fold in range(k):
  validation_data = data[num_validation_samples * fold:num_validation_samples * (fold + 1)] # Selects the validation data partition
  training_data = data[:num_validation_samples * fold] + data[num_validation_samples * (fold + 1):] # Uses the remainder of the data as training data. Note that the + operator is list concatenation, not summation.
  model = get_model() # Creates a brand-new instance of the model (untrained)
  model.train(training_data)
  validation_score = model.evaluate(validation_data)
  validation_scores.append(validation_score)

validation_score = np.average(validation_scores)  # Validation score: average of the validation scores of the k folds
model = get_model()
model.train(data) # Trains the final model on all nontest data available
test_score = model.evaluate(test_data)

**4.2 K-fold corss-validation**

With this approach, you split your data into K partitions of equal size. For each partition i, train a model on the remaining K – 1 partitions, and evaluate it on partition i. Your final score is then the averages of the K scores obtained. This method is helpful when the performance of your model shows significant variance based on your traintest split. Like hold-out validation, this method doesn’t exempt you from using a distinct validation set for model calibration.

# **4.2.2 - Things To Keep in Mind**



***When choosing an evaluation protocol, consider the followings:***

*   **Data representativeness** — You want both your training set and test set to be representative of the data at hand. For instance, if you’re trying to classify images of digits, and you’re starting from an array of samples where the samples are ordered by their class, taking the first 80% of the array as your training set and the remaining 20% as your test set will result in your training set containing only classes 0–7, whereas your test set contains only classes 8–9. This seems like a ridiculous mistake, but it’s surprisingly common. For this reason, **you usually should randomly shuffle your data before splitting it into training and test sets**.

*   **The arrow of time** — If you’re trying to predict the future given the past (for example, tomorrow’s weather, stock movements, and so on), **you should not randomly shuffle your data before splitting it**, because doing so will create a temporal leak: your model will effectively be trained on data from the future. In such situations, you should always make sure all data in your test set is posterior to the data in the training set.

*   **Redundancy in your data** — If some data points in your data appear twice (fairly common with real-world data), then shuffling the data and splitting it into a training set and a validation set will result in redundancy between the training and validation sets. In effect, you’ll be testing on part of your training data, which is the worst thing you can do! **Make sure your training set and validation set are disjoint**.

***While data pre-processing, consider the followings:***

*   **Vectorization** - All inputs and targets in a neural network must be tensors of floating-point data (or, in specific cases, tensors of integers). Whatever data you need to process—sound, images, text—you must first turn into tensors, a step called data vectorization. For instance, in the two previous text-classification examples, we started from text represented as lists of integers (standing for sequences of words), and we used one-hot encoding to turn them into a tensor of float32 data. In the examples of classifying digits and predicting house prices, the data already came in vectorized form, so you were able to skip this step.

*   **Value normalization** - In the digit-classification example, you started from image data encoded as integers in the 0–255 range, encoding grayscale values. Before you fed this data into your network, you had to cast it to float32 and divide by 255 so you’d end up with floatingpoint values in the 0–1 range. Similarly, when predicting house prices, you started from features that took a variety of ranges—some features had small floating-point values, others had fairly large integer values. Before you fed this data into your network, you had to normalize each feature independently so that it had a standard deviation of 1 and a mean of 0. **In general, it isn’t safe to feed into a neural network data that takes relatively large values** (for example, multidigit integers, which are much larger than the initial values taken by the weights of a network) or data that is heterogeneous. Doing so can trigger large gradient updates that will prevent the network from converging. A common prcatice would be:

  *   **Normalize each feature independently to have a mean of 0** : *x -= x.mean(axis = 0)*
  *   **Normalize each feature independently to have a standard deviation of 1** : *x /= x.std(axis = 0)*

*   **Handling missing values** - You may sometimes have missing values in your data. For instance, in the house-price example, the first feature (the column of index 0 in the data) was the per capita crime rate. What if this feature wasn’t available for all samples? You’d then have missing values in the training or test data. In general, with neural networks, **it’s safe to input missing values as 0, with the condition that 0 isn’t already a meaningful value**. The network will learn from exposure to the data that the value 0 means missing data and will start ignoring the value.



# **4.4 - Overfitting & Underfitting: Regularization**

***There are a number of common techniques to fight the overfitting:***

*   **Reducing the network’s size** - The simplest way to prevent overfitting is to reduce the size of the model: the number of learnable parameters in the model (which is determined by the number of layers and the number of units per layer). Intuitively, a model with more parameters has more memorization capacity and therefore can easily learn a perfect dictionary-like mapping between training samples and their targets—a mapping without any generalization power. *The general workflow to find an appropriate model size is to start with relatively few layers and parameters, and increase the size of the layers or add new layers until you see diminishing returns with regard to validation loss.*

*   **Adding weight regularization** - Like the principle of *Occam’s razor*, his idea also applies to the models learned by neural networks: given some training data and a network architecture, multiple sets of weight values (multiple models) could explain the data. Simpler models are less likely to overfit than complex ones.

  ```
  from keras import regularizers
  model = models.Sequential()
  model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
  activation='relu', input_shape=(10000,)))
  model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  ```
*   **Adding dropout** - Dropout is one of the most effective and most commonly used regularization techniques for neural networks. Dropout, applied to a layer, consists of randomly dropping out (setting to zero) a number of output features of the layer during training. **The core idea is that introducing noise in the output values of a layer can break up happenstance patterns that aren’t significant**.
  
  ```
  # Adding dropout to the IMDB network
  model = models.Sequential()
  model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))
  ```





# **4.5 - The universal workflow of machine learning**

**Defining the problem and assembling a dataset** - What will your input data be? What are you trying to predict? You can only learn to predict something if you have available training data. After making sure of the answers to these questions, time to answer even more: What type of problem are you facing? Is it binary classification? Multiclass classification? Scalar regression? Vector regression? Multiclass, multilabel classification? Something else, like clustering, generation, or reinforcement learning?

You can’t move to the next stage until you know what your inputs and outputs are, and what data you’ll use.

**Choosing a measure of success** - To control something, you need to be able to observe it. To achieve success, you must define what you mean by success; Your metric for success will guide the choice of a loss function: what your model will optimize. For balanced-classification problems, where every class is equally likely, accuracy and area under the receiver operating characteristic curve (ROC AUC) are common metrics. For class-imbalanced problems, you can use precision and recall. For ranking problems or multilabel classification, you can use mean average precision.

**Deciding on an evaluation protocol** - Once you know what you’re aiming for, you must establish how you’ll measure your current progress. We’ve previously reviewed three common evaluation protocols:
*   Maintaining a hold-out validation set—The way to go when you have plenty of
data
*   Doing K-fold cross-validation—The right choice when you have too few samples
for hold-out validation to be reliable
*   Doing iterated K-fold validation—For performing highly accurate model evaluation when little data is available

**Preparing your data** - First, you should format your data in a way that can be fed into a machine-learning model. 
*   As you saw previously, your data should be formatted as tensors.
*   The values taken by these tensors should usually be scaled to small values: for example, in the [-1, 1] range or [0, 1] range.
*   If different features take values in different ranges (heterogeneous data), then the data should be normalized.
*   You may want to do some feature engineering, especially for small-data problems.

**Developing a model that does better than a baseline** - Your goal at this stage is to achieve statistical power : that is, to develop a small model
that is capable of beating a dumb baseline. Note that it’s not always possible to achieve statistical power. If you can’t beat a random baseline after trying multiple reasonable architectures, it may be that the answer to the question you’re asking isn’t present in the input data. Here are some right last-layer activation and loss function for your model:

*   **Binary classification** -> sigmoid(Activation) &  binary_crossentropy(Loss)
*   **Multiclass, single-label classification** -> softmax(Activation) & categorical_crossentropy(Loss)
*   **Multiclass, multilabel classification** -> sigmoid(Activation) & binary_crossentropy(Loss)
*   **Regression to arbitrary values** -> None(Activayion) & MSE(Loss)
*   **Regression to values between 0 and 1** -> sigmoid(Activation)  & (MSE or binary_crossentropy)(Loss)

**Scaling up: developing a model that overfits** - Once you’ve obtained a model that has statistical power, the question becomes, is your model sufficiently powerful? For instance, a network with a single hidden layer with
two units would have statistical power on MNIST but wouldn’t be sufficient to solve the problem well. Remember that the universal tension in machine learning is between optimization and generalization. To figure out how big a model you’ll need, you must develop a model that overfits. This is fairly easy:

1.   Add layers.
2.   Make the layers bigger.
3.   Train for more epochs.

**Regularizing your model and tuning your hyperparameters** - This step will take the most time: you’ll repeatedly modify your model, train it, evaluate
on your validation data (not the test data, at this point), modify it again, and
repeat, until the model is as good as it can get. These are some things you should try:

*   Add dropout
*   Try different architectures: add or remove layers
*   Add L1 and/or L2 regularization
*   Try different hyperparameters (such as the number of units per layer or the
learning rate of the optimizer) to find the optimal configuration
*   Optionally, iterate on feature engineering: add new features, or remove features that don’t seem to be informative.
